In [42]:
import pandas as pd

In [43]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
df = pd.read_csv('/content/gdrive/MyDrive/AdvancedML/IoT Network Intrusion Dataset.csv')
df.head

<bound method NDFrame.head of                                           Flow_ID           Src_IP  Src_Port  \
0        192.168.0.13-192.168.0.16-10000-10101-17     192.168.0.13     10000   
1         192.168.0.13-222.160.179.132-554-2179-6  222.160.179.132      2179   
2          192.168.0.13-192.168.0.16-9020-52727-6     192.168.0.16     52727   
3          192.168.0.13-192.168.0.16-9020-52964-6     192.168.0.16     52964   
4       192.168.0.1-239.255.255.250-36763-1900-17      192.168.0.1     36763   
...                                           ...              ...       ...   
625778   192.168.0.24-210.89.164.90-56112-8043-17     192.168.0.24     56112   
625779    192.168.0.13-222.131.171.244-554-4570-6  222.131.171.244      4570   
625780     192.168.0.13-192.168.0.16-9020-52739-6     192.168.0.16     52739   
625781     192.168.0.13-192.168.0.16-9020-49784-6     192.168.0.13      9020   
625782   192.168.0.13-192.168.0.16-10000-10101-17     192.168.0.13     10000   

         

In [45]:
df.shape

(625783, 86)

In [46]:
# drop columns with string values and values which are not fitting in float64
df_copy = df.drop(["Flow_ID","Src_IP","Dst_IP","Timestamp", "Cat","Sub_Cat","Label", "Flow_Byts/s", "Flow_Pkts/s"], axis=1)
df_copy.shape

(625783, 77)

In [47]:
# Binary Classification using Naive Bayes
# Defining our features and target columns

features = df[df_copy.columns]
target = df[df.columns[-3]] 
print(features.shape)
print(target.shape)

(625783, 77)
(625783,)


In [48]:
target

0         Anomaly
1         Anomaly
2         Anomaly
3         Anomaly
4         Anomaly
           ...   
625778    Anomaly
625779    Anomaly
625780    Anomaly
625781     Normal
625782    Anomaly
Name: Label, Length: 625783, dtype: object

In [49]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target = le.fit_transform(target)

In [50]:
target

array([0, 0, 0, ..., 0, 1, 0])

In [51]:
target = target.reshape(-1, 1)
target

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [52]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
minMax= MinMaxScaler()
features = minMax.fit_transform(features)
sc = StandardScaler()
features = sc.fit_transform(features)



In [53]:
# Training Model

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=1)

print(X_train.shape) 
print(X_test.shape) 

print(y_train.shape) 
print(y_test.shape)

(438048, 77)
(187735, 77)
(438048, 1)
(187735, 1)


In [54]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [55]:
y_pred = gnb.predict(X_test)

In [56]:
from sklearn import metrics 
print("Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Naive Bayes model accuracy(in %): 53.54888539696914


In [57]:
#Multi class classification using Naive Bayes
features2 = features
target2 = df[df.columns[-1]] 
print(features2.shape)
print(target2.shape)

(625783, 77)
(625783,)


In [58]:
target2

0             Mirai-Ackflooding
1               DoS-Synflooding
2                  Scan Port OS
3         Mirai-Hostbruteforceg
4         Mirai-Hostbruteforceg
                  ...          
625778       Mirai-UDP Flooding
625779          DoS-Synflooding
625780             Scan Port OS
625781                   Normal
625782       Mirai-UDP Flooding
Name: Sub_Cat, Length: 625783, dtype: object

In [60]:
target2 = le.fit_transform(target2).reshape(-1, 1)
target2

array([[2],
       [0],
       [8],
       ...,
       [8],
       [6],
       [5]])

In [61]:
# Training Model

from sklearn.model_selection import train_test_split 
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, target2, test_size=0.3, random_state=1)

print(X_train2.shape) 
print(X_test2.shape) 

print(y_train2.shape) 
print(y_test2.shape)

(438048, 77)
(187735, 77)
(438048, 1)
(187735, 1)


In [164]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB(priors=None, var_smoothing=1e-03) 
gnb.fit(X_train2, y_train2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(var_smoothing=0.001)

In [165]:
y_pred2 = gnb.predict(X_test2)

In [166]:
from sklearn import metrics 
print("Naive Bayes Multi Class Classification model accuracy(in %):", metrics.accuracy_score(y_test2, y_pred2)*100)

Naive Bayes Multi Class Classification model accuracy(in %): 48.68404932484619


In [65]:
# Desision Trees
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [66]:
# Binary class Classification with decision trees
# We will use binary classification variables in this case i.e x_train, y_train etc
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred_tree = clf.predict(X_test)

In [67]:
# Model Accuracy
print("Accuracy Decision Tree Binary class classification:",metrics.accuracy_score(y_test, y_pred_tree))

Accuracy Decision Tree Binary class classification: 0.9991477348390018


In [136]:
# Multi class Classification with decision trees
# We will use multi class classification variables in this case i.e x_train2, y_train2 etc
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(max_depth=18, max_features=74)

# Train Decision Tree Classifer
clf = clf.fit(X_train2,y_train2)

#Predict the response for test dataset
y_pred_tree2 = clf.predict(X_test2)

In [137]:
# Model Accuracy with multi class classification
print("Accuracy Decision Tree multi class classification:",metrics.accuracy_score(y_test2, y_pred_tree2))

Accuracy Decision Tree multi class classification: 0.7864915971981783
